**Exercise 2**
<br/>
Design and implement a simple digital certificate framework. Your framework should
allow to create a certificate chain and validate it. (Suppose there are 3 CAs to form
a certificate chain.)


In [1]:
!pip install rsa
!pip install cryptography

In [6]:
from datetime import datetime, timedelta
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric.padding import PKCS1v15


# Class Entity represent to an Entity with private key and public key
class Entity:
    # init function, generate (private key, public key)
    def __init__(self, name: str, public_exponent=65537, key_size=2048):
        self.name = name
        self.__keys = rsa.generate_private_key(public_exponent, key_size)

    # as public key is public, this function will return public key if needed
    def get_public_key(self):
        return self.__keys.public_key()


# represent for an Certificate Authenticator
# Certificate Authenticator class is inherited from Entity class
class CertificateAuthority(Entity):
    # init function, save the name, generate private, public keys
    def __init__(self, name: str):
        # init name, keys for Certificate Authority entity (public & private keys)
        super().__init__(name)

    # generate signature, sign by itself
    def generate_signature(self, subject_name, public_key):
        # requestor name
        subject = x509.Name(
            [
                x509.NameAttribute(NameOID.ORGANIZATION_NAME, subject_name),
                x509.NameAttribute(NameOID.COMMON_NAME, subject_name),
            ]
        )

        # CA name
        issuer = x509.Name(
            [
                x509.NameAttribute(NameOID.ORGANIZATION_NAME, self.name),
                x509.NameAttribute(NameOID.COMMON_NAME, self.name),
            ]
        )

        # form a certificate object to the builder
        certificate = (
            x509.CertificateBuilder()
            .subject_name(subject)
            .issuer_name(issuer)
            .public_key(public_key)
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.utcnow())
            .not_valid_after(datetime.utcnow() + timedelta(days=365))
            .sign(self._Entity__keys, hashes.SHA256())
        )
        return certificate

    # verify certificate based on its public key
    def verify_certificate(self, certificate):
        try:
            # if validated: return True
            # else: throw error
            self.get_public_key().verify(
                certificate.signature,
                certificate.tbs_certificate_bytes,
                PKCS1v15(),
                certificate.signature_hash_algorithm,
            )
            # passed authentication
            return True
        except Exception as e:
            print(f"Error: {e}")
            # authentication failed
            return False

# return cert as PEM format
def get_certificate(cert):
    return cert.public_bytes(serialization.Encoding.PEM).decode()


# initial root CA & intermediate CA
root_ca = CertificateAuthority("Root CA")
# root cert generate by itself
root_ca_cert = root_ca.generate_signature("Root CA", root_ca.get_public_key())
print(f"Root CA certificate: \n {get_certificate(root_ca_cert)}")

intermediate_ca = CertificateAuthority("Intermediate CA")
# intermediate cert generated by root CA
intermediate_ca_cert = root_ca.generate_signature(
    "Intermediate CA", intermediate_ca.get_public_key()
)
print(f"\nIntermediate CA certificate: \n {get_certificate(intermediate_ca_cert)}")

alice = Entity("Alice")
# Alice cert generated by intermediate CA
alice_certificate = intermediate_ca.generate_signature("Alice", alice.get_public_key())
print(f"\nAlice certificate: \n {get_certificate(alice_certificate)}")
# validate Alice's certificate
if intermediate_ca.verify_certificate(alice_certificate):
    print("Verified Alice certificate")
else:
    print("Alice certificate is unauthorized")

Root CA certificate: 
 -----BEGIN CERTIFICATE-----
MIIC1DCCAbygAwIBAgIUX6Sf0xm3EPWWd0K7GEthRpz4sBswDQYJKoZIhvcNAQEL
BQAwJDEQMA4GA1UECgwHUm9vdCBDQTEQMA4GA1UEAwwHUm9vdCBDQTAeFw0yNDEy
MTQwMjI3NTRaFw0yNTEyMTQwMjI3NTRaMCQxEDAOBgNVBAoMB1Jvb3QgQ0ExEDAO
BgNVBAMMB1Jvb3QgQ0EwggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQCV
xC/GGCzeQS9MQscH514a+lQwD/Kxl+eBVJ+KXH8djakzt1fnxtOtDyw/cmZeoFRJ
2uw/y0gfzSTnfDYmCLhKuAttu8hpjMER/C84mSHDOZBXe6Iac4iH1gtBY4uwVsnP
zxzQB45BJDegZuqyxgtRBqEqDnGtHmc2TZ70sIq3xowXohL3KBv1rSlfCkXO32WK
5YKjRN0uLaJaaHf6+jmQEQtGQh8F86xnL23js4qkoAL2g/vg1l6wHR8gpVhOMu4o
JakR71GOs0heUy53XB3+Q0atfYRO5pnsg8aCA7WI/E0AwapYmFNhyNKGiJmDWcut
WdxUq1r2WWDX2Q/yXjkpAgMBAAEwDQYJKoZIhvcNAQELBQADggEBAH64FYxNA9a0
GDnegoAajBk+8sq/yNL9h9QY4yFrE/48XozeKqDW0WKVsm/xzNYvjMRxpJ4As0SJ
cZAWsSUAzx+FMKAF2Ba7Ajo1rnfI2MOByWhT8Szd5sd88Lkyo3G8z9uj2BW+zIGL
Hdwq+VXfHvX9aYWobKpkcjoKz7sqHU9YwtMYFAFDeeEjYU77VHIjDiEH0kXzJz2w
z/SxyDhnKQYDOdwncG+bywqkX9sZT4BuWLvULdA3KkfR2BBMhCWlvq0oRNlxKD05
zt1miIXr/tN7Cpf89/C8WB5GPUS6nuTZ4S2lOWF